### Load the libraries/modules

In [1]:
import os, sys, copy, time
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import statsmodels.api as sm
import functions as func
import warnings
warnings.filterwarnings('ignore')

### Load the necessary data

In [2]:
# Load all the necessary data
nifty50_bbc = pd.read_csv('./../data/nifty50_before_before_covid.csv')
nifty50_bc = pd.read_csv('./../data/nifty50_before_covid.csv')
nifty50_ac = pd.read_csv('./../data/nifty50_after_covid.csv')
firms_bbc = pd.read_csv('./../data/map_data_before_before_covid.csv')
firms_bc = pd.read_csv('./../data/map_data_before_covid.csv')
firms_ac = pd.read_csv('./../data/map_data_after_covid.csv')
banks_vulnerability = pd.read_csv('./../data/banks_vulnerability.csv', index_col=0)
banks_vulnerability.columns = ['symbol', 'banker_name', 'type', 'beta', 'mes', 'vulnerable']
banks_vulnerability['banker_name'] = banks_vulnerability['banker_name'].apply(lambda x: x.upper())
banks_firms_connection = pd.read_csv('./../data/bank_firm_connection_filter.csv')
banks_firms_connection.drop(['bnkhist_date'], axis = 1, inplace = True)


### Date-time column to datetime object

In [3]:
nifty50_bbc['Date'] = pd.to_datetime(nifty50_bbc['Date'])
nifty50_bc['Date'] = pd.to_datetime(nifty50_bc['Date'])
nifty50_ac['Date'] = pd.to_datetime(nifty50_ac['Date'])
nifty50_bbc.index = nifty50_bbc['Date']
nifty50_bc.index = nifty50_bc['Date']
nifty50_ac.index = nifty50_ac['Date']
firms_bbc['co_stkdate'] = pd.to_datetime(firms_bbc['co_stkdate'])
firms_bc['co_stkdate'] = pd.to_datetime(firms_bc['co_stkdate'])
firms_ac['co_stkdate'] = pd.to_datetime(firms_ac['co_stkdate'])
firms_bbc.index = firms_bbc['co_stkdate']
firms_bc.index = firms_bc['co_stkdate']
firms_ac.index = firms_ac['co_stkdate']

In [4]:
# ### Demo
# # Demo for company: 20 MICRONS LTD.
# firm_cp = firms_bc.loc[firms_bc['company_name'] == '20 MICRONS LTD.']['bse_closing_price']
# # firm_cp.reset_index(drop = True, inplace = True)
# merged_CP = func.merging_CP(firm_cp, nifty50_bc['Close'])
# merged_CP.head()

# reg_data = func.making_data_reg(merged_CP)
# reg_data.dropna(inplace=True)
# reg_data.head()
# residuals = func.reg_residuals(reg_data.drop(['y'], axis=1).values, reg_data['y'].values)
# df = pd.DataFrame({'firm_return':reg_data['y'],'residuals':residuals, 
#                 'w':func.firm_specific_return(residuals)})
# df.head()
# func.DUVOL(df)
# n = 1
# p = 0
# if n in [0,1] or p in [0,1]:
#     print(n,p)

### Calculating crash risk

Running over all firms across time period:

Limitation: All firms do not have much data.
- Run for those firms which have exact 120 days data available.
- NSE/BSE (Which have more data)

In [5]:
# Printing the shape of the data
print(f'Shape of the mapping data is (Initial stage of corona): {firms_bbc.shape}')
print(f'Shape of the mapping data is (Before corona): {firms_bc.shape}')
print(f'Shape of the mapping data is (After corona): {firms_ac.shape}')

Shape of the mapping data is (Initial stage of corona): (278641, 6)
Shape of the mapping data is (Before corona): (278817, 6)
Shape of the mapping data is (After corona): (277338, 6)


In [6]:
# Filtered out by exchange center which have atleast 100 days data
df_nse = firms_bc.dropna(thresh=2, axis=0).groupby(by = ['company_name'])['nse_closing_price'].count().reset_index()
df_nse = df_nse[df_nse.nse_closing_price > 35]
df_bse = firms_bc.dropna(thresh=2, axis=0).groupby(by = ['company_name'])['bse_closing_price'].count().reset_index()
df_bse = df_bse[df_bse.bse_closing_price > 35]

In [7]:
# Separate out by exchange center
company_name_nse = df_nse['company_name'].values
company_name_bse = []
for company in df_bse['company_name'].values:
    if company not in company_name_nse:
        company_name_bse.append(company)

In [8]:
result_crash_risk_bbc = []
firms_singular_bbc = []
# NSE: Before of Before COVID
for firm in tqdm(company_name_nse):
    try:
        firm_cp = firms_bbc.loc[firms_bbc['company_name'] == firm]['nse_closing_price']
        market_cp = nifty50_bbc['Close']
        result = func.crash_risk(firm_cp, market_cp)
        result_crash_risk_bbc.append(('NSE', firm, -1, result[0], result[1], result[2]))
    except:
        firms_singular_bbc.append(firm)

# BSE: Before of Before COVID
for firm in tqdm(company_name_bse):
    try:
        firm_cp = firms_bbc.loc[firms_bbc['company_name'] == firm]['bse_closing_price']
        market_cp = nifty50_bbc['Close']
        result = func.crash_risk(firm_cp, market_cp)
        result_crash_risk_bbc.append(('BSE', firm, -1, result[0], result[1], result[2]))
    except:
        firms_singular_bbc.append(firm)

result_crash_risk_bbc = pd.DataFrame(result_crash_risk_bbc, columns=['exchange','company_name','time','count','nskew','duvol'])
result_crash_risk_bbc.to_csv('./../data/result_jan_june_19.csv', index = False)

  0%|          | 0/1384 [00:00<?, ?it/s]

  0%|          | 0/1121 [00:00<?, ?it/s]

In [9]:
result_crash_risk_bc = []
firms_singular_bc = []
# NSE: Before COVID
for firm in tqdm(company_name_nse):
    try:
        firm_cp = firms_bc.loc[firms_bc['company_name'] == firm]['nse_closing_price']
        market_cp = nifty50_bc['Close']
        result = func.crash_risk(firm_cp, market_cp)
        result_crash_risk_bc.append(('NSE', firm, 0, result[0], result[1], result[2]))
    except:
        firms_singular_bc.append(firm)

# BSE: Before COVID
for firm in tqdm(company_name_bse):
    try:
        firm_cp = firms_bc.loc[firms_bc['company_name'] == firm]['bse_closing_price']
        market_cp = nifty50_bc['Close']
        result = func.crash_risk(firm_cp, market_cp)
        result_crash_risk_bc.append(('BSE', firm, 0, result[0], result[1], result[2]))
    except:
        firms_singular_bc.append(firm)

result_crash_risk_bc = pd.DataFrame(result_crash_risk_bc, columns=['exchange','company_name','time','count','nskew','duvol'])
# result_crash_risk_bc

  0%|          | 0/1384 [00:00<?, ?it/s]

  0%|          | 0/1121 [00:00<?, ?it/s]

In [10]:
result_crash_risk_ac = []
firms_singular_ac = []
# NSE: During COVID
for firm in tqdm(company_name_nse):
    try:
        firm_cp = firms_ac.loc[firms_ac['company_name'] == firm]['nse_closing_price']
        market_cp = nifty50_ac['Close']
        result = func.crash_risk(firm_cp, market_cp)
        result_crash_risk_ac.append(('BSE', firm, 1, result[0], result[1], result[2]))
    except:
        firms_singular_ac.append(firm)
# BSE: During COVID
for firm in tqdm(company_name_bse):
    try:
        firm_cp = firms_ac.loc[firms_ac['company_name'] == firm]['bse_closing_price']
        market_cp = nifty50_ac['Close']
        result = func.crash_risk(firm_cp, market_cp)
        result_crash_risk_ac.append(('BSE', firm, 1, result[0], result[1], result[2]))
    except:
        firms_singular_ac.append(firm)
result_crash_risk_ac = pd.DataFrame(result_crash_risk_ac, columns=['exchange','company_name','time','count','nskew','duvol'])

  0%|          | 0/1384 [00:00<?, ?it/s]

  0%|          | 0/1121 [00:00<?, ?it/s]

In [11]:
final_result = pd.concat([result_crash_risk_bc, result_crash_risk_ac], axis=0)
final_result.sort_values(by = ['company_name'], inplace = True)
final_result = final_result[~final_result['company_name'].isin(firms_singular_bc)]
final_result = final_result[~final_result['company_name'].isin(firms_singular_ac)]

In [12]:
banks_vulnerability['vulnerable'] = banks_vulnerability['vulnerable'].astype(int)

In [13]:
final_result = pd.merge(final_result, banks_firms_connection, on = 'company_name')
final_result = pd.merge(final_result, banks_vulnerability, on = 'banker_name')
final_result = final_result.loc[:,['symbol','banker_name','co_code','company_name','exchange','beta','mes','vulnerable','type','time','count','nskew','duvol']]
final_result.head()

,symbol,banker_name,co_code,company_name,exchange,beta,mes,vulnerable,type,time,count,nskew,duvol
0,SBIN.NS,STATE BANK OF INDIA,11,20 MICRONS LTD.,NSE,0.247401,0.001617,1,Public,0,0,-0.626466,-0.655800
1,SBIN.NS,STATE BANK OF INDIA,11,20 MICRONS LTD.,BSE,0.247401,0.001617,1,Public,1,-1,-0.814185,-0.664436
2,SBIN.NS,STATE BANK OF INDIA,576805,A & M JUMBO BAGS LTD.,BSE,0.247401,0.001617,1,Public,1,0,0.187605,0.257741
3,SBIN.NS,STATE BANK OF INDIA,576805,A & M JUMBO BAGS LTD.,NSE,0.247401,0.001617,1,Public,0,0,0.136015,0.074015
4,SBIN.NS,STATE BANK OF INDIA,389178,A 2 Z INFRA ENGG. LTD.,BSE,0.247401,0.001617,1,Public,1,-3,-0.763392,-0.171274


In [14]:
cr_bbc = result_crash_risk_bbc[['company_name','count','nskew','duvol']]
cr_bbc.columns = ['company_name','ic_count','ic_nskew','ic_duvol']
final_result = pd.merge(final_result, cr_bbc, on = 'company_name')
final_result = final_result[~final_result['company_name'].isin(firms_singular_bbc)]
print(final_result.shape)

(4238, 16)


In [15]:
final_result.dropna(inplace = True)

In [16]:
final_result.to_csv('./../data/final_table.csv', index = False)

In [18]:
np.sum(final_result == np.nan)

symbol          0
banker_name     0
co_code         0
company_name    0
exchange        0
beta            0
mes             0
vulnerable      0
type            0
time            0
count           0
nskew           0
duvol           0
ic_count        0
ic_nskew        0
ic_duvol        0
dtype: int64

In [19]:
final_result.shape

(4180, 16)